# 📡 06 - Model Inference to Alerts

This notebook uses a trained anomaly detection model to infer risk scores from sensor feature data, 
filters high-risk records (anomaly = 1), and writes alerts into a timestamped Delta table.

**Steps:**
- Read predictions from the latest model
- Filter only anomalous records
- Write them to a time-versioned table for downstream alerting or monitoring


In [0]:
from pyspark.sql.functions import col, current_timestamp, lit
from datetime import datetime

timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")

# Load full prediction output
predictions = spark.read.table("arao.aerodemo.prediction_results")

# Fix types (cast everything Delta expects)
predictions_casted = (
    predictions
    .withColumn("timestamp", col("timestamp").cast("string"))
    .withColumn("aircraft_id", col("aircraft_id").cast("string"))
    .withColumn("engine_rpm", col("engine_rpm").cast("int"))
    .withColumn("capacity", col("capacity").cast("int"))
    .withColumn("range_km", col("range_km").cast("int"))
    .withColumn("predicted_anomaly", col("predicted_anomaly").cast("long"))
)

# Add metadata
predictions_with_meta = (
    predictions_casted
    .withColumn("alert_generated_at", current_timestamp())
    .withColumn("batch_id", lit(timestamp_str))
)

# Recreate alert table
spark.sql("DROP TABLE IF EXISTS arao.aerodemo.anomaly_alerts")

predictions_with_meta.limit(0).write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("arao.aerodemo.anomaly_alerts")

# Filter and insert alerts
alerts = predictions_with_meta.filter(col("predicted_anomaly") == 1)

alerts.write \
    .format("delta") \
    .mode("append") \
    .saveAsTable("arao.aerodemo.anomaly_alerts")

print(f"✅ Alert table recreated and {alerts.count()} alerts written with batch ID {timestamp_str}")

### 📌 Notes:
- `alert_generated_at`: records the timestamp of alert generation
- `batch_id`: uniquely identifies the batch for traceability or rollback
- This table can be used for integration with dashboards, alerting systems, or compliance reporting
